1\)

Importing Data

In [4]:
import numpy as np

from sklearn import datasets
iris = datasets.load_iris()
X = iris.data[:,0:4] ## from this only take features 0,1,2,3
Y = iris.target

print(X.shape)
print(Y)

(150, 4)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [17]:
def PGauss(mu, sig, x):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.) + 1e-300) )

def processing_for_NB(X, Y):
    U = np.zeros((X.shape[1], 3))
    S = np.zeros((X.shape[1], 3))
    PClass = []

    for c in range(3):
        PClass.append(np.array(np.where(Y==c)).shape[1]/Y.shape[0])
        for f in range(X.shape[1]):
            U[f,c] = X[np.where(Y==c),f].mean()
            S[f,c] = X[np.where(Y==c),f].std()

    return PClass, U, S
    
PClass, U, S = processing_for_NB(X,Y)
print(PClass)
print(U)
print(S)

def NB(X, PClass, U, S, num_classes = 3):
    Y = np.ones((X.shape[0], num_classes))
    for i in range(Y.shape[0]): #0-149
        for c in range(Y.shape[1]): #0-3
            Y[i,c] *= PClass[c]
            for f in range(X.shape[1]): #0-4
                Y[i,c] *= PGauss(U[f,c], S[f,c], X[i, f])
    return Y

def argmax(Z):
    maxz = max(Z)
    for j in range(len(Z)):
        Z[j] = 1.0 if Z[j]>=maxz else 0
    return(Z)

Y = NB(X, PClass, U, S)
print(Y)
        
            




[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
[[5.006 5.936 6.588]
 [3.428 2.77  2.974]
 [1.462 4.26  5.552]
 [0.246 1.326 2.026]]
[[0.34894699 0.51098337 0.62948868]
 [0.37525458 0.31064449 0.31925538]
 [0.17191859 0.46518813 0.54634787]
 [0.10432641 0.19576517 0.27188968]]
[[2.68329763e-001 2.24258759e-018 2.42603215e-025]
 [1.41221715e-001 1.31671002e-017 4.21523358e-025]
 [1.09824774e-001 7.25350822e-019 3.26701295e-026]
 [1.02374648e-001 9.23879732e-018 3.84524663e-025]
 [2.55114962e-001 7.11778741e-019 9.35192567e-026]
 [1.03079187e-002 9.45939417e-016 2.30280221e-022]
 [1.38449452e-001 9.39617416e-018 4.77174646e-025]
 [2.94293119e-001 1.18400334e-017 1.03746108e-024]
 [2.33135985e-002 1.35206026e-018 3.56924577e-026]
 [7.96258456e-002 1.67785117e-018 1.22216142e-025]
 [1.19984702e-001 3.23534657e-018 7.72989368e-025]
 [1.83594423e-001 1.87321905e-017 1.69134397e-024]
 [5.14281532e-002 4.03823197e-019 2.14887251e-026]
 [9.19306027e-004 4.37811957e-022 2.17592235e-